In [1]:
# !pip uninstall -y tensorflow
# !pip uninstall -y tensorflow_hub
# !pip uninstall -y tensorflow_text

# !pip install tensorflow
# !pip install tensorflow_hub
# !pip install tensorflow_text


In [7]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.1
    Uninstalling pip-24.1.1:
      Successfully uninstalled pip-24.1.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install tensorflow
!pip install tensorflow_hub


  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.12.2
    Uninstalling typing-extensions-4.12.2:
      Successfully uninstalled typing-extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.31 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.8.2 requires typing-extensions>=4.6.1; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.20.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
openai 1.35.13 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
You should consider upgrading via the '/Users/rohitsunny/.pyenv/versions/3.8.12

In [8]:
pip install tensorflow-text


ERROR: Could not find a version that satisfies the requirement tensorflow-text (from versions: none)
ERROR: No matching distribution found for tensorflow-text
Note: you may need to restart the kernel to use updated packages.


In [1]:
python --version


NameError: name 'python' is not defined

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

ModuleNotFoundError: No module named 'tensorflow_text'

In [ ]:
import pandas as pd

df = pd.read_csv("spam.csv")


df.head()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.groupby('Category').describe()

In [ ]:
df['Category'].value_counts()

In [ ]:
747/4825

In [ ]:
##As the data is imbalance we need to use some techniques that can covercome this we willl be using undersampling

In [ ]:
df_spam = df[df["Category"] == 'spam']
df_spam.shape

In [ ]:
df_ham = df[df['Category'] == 'ham']
df_ham.shape

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

In [ ]:
df_balance = pd.concat([df_spam, df_ham_downsampled])
df_balance.shape

In [ ]:
df_balance['Category'].value_counts()

In [ ]:
df_balance.sample(5)

In [ ]:
#creating a binary column for spam
df_balance['spam'] = df_balance['Category'].apply(lambda x: 1 if x=='spam' else 0 )

In [ ]:
df_balance.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balance['Message'], df_balance['spam'], stratify = df_balance['spam'])

In [ ]:
X_train.head(5)

In [ ]:
X_test.head(5)

In [ ]:
y_train

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  embeding =  bert_encoder(preprocessed_text)['pooled_output']
  return embeding.numpy()


get_sentence_embeding(["500$ discount. hurry up",
                       "Bhavin, are you up for a vollebal game tonight?"])



In [ ]:
e = get_sentence_embeding([
    'banana',
    'grapes',
    'mango',
    'jeff bezos',
    'elon musk',
    'bill gates'
])

In [ ]:
e

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0]], [e[1]])

In [ ]:
cosine_similarity([e[0]], [e[3]])

In [ ]:
cosine_similarity([e[3]], [e[4]])

In [ ]:
# Example usage
sentences = [
    "I love playing football.",
    "What is the capital of France?",
    "Turn off the lights.",
    "BERT stands for Bidirectional Encoder Representations from Transformers.",
    "Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence...",
    "A: How are you? B: I'm fine, thank you!",
    "This is a lightweight and portable laptop with a 15-inch screen, 8GB RAM, and 256GB SSD.",
    "Stock markets reach all-time high.",
    "The book was fantastic! I couldn't put it down.",
    "Just had an amazing coffee at the new cafe!"
]

embedings = get_sentence_embeding(sentences)
print(embedings)

In [ ]:
embedings = get_sentence_embeding(sentences[:9])


In [ ]:

cosine_similarity([embedings[0]], [embedings[8]])

In [ ]:
#https://becominghuman.ai/sequential-vs-functional-model-in-keras-20684f766057

#bert layers



text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name = 'text')

preprocessed_text = bert_preprocess(text_input)

outputs = bert_encoder(preprocessed_text)

#neural networks layers

l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])

l = tf.keras.layers.Dense(1, activation = 'sigmoid', name='output')(l)

#contruct a final model

model = tf.keras.Model(inputs=[text_input], outputs=[l])


In [ ]:
model.summary()

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics =METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()

In [ ]:
import numpy as np

y_pred = np.where(y_pred > 0.5,1,0)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report # Fixed typo here

cm = confusion_matrix(y_test, y_pred)
cm


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm,annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_pred))